# IXXAT USB-to-CAN V2 Setup Guide (READ THIS to make sure everything and try steps 5/6 are working before running code)

## 1. Check if Driver is Already Installed

First, check if the driver is already loaded:

```bash
lsmod | grep ix_usb_can
```

If you see output, the driver is loaded. You can also check if the interface exists:

```bash
ip link show | grep can
```

## 2. Download and Build the Driver (if needed)

If the driver isn't loaded, follow these steps:

```bash
# Create a directory for the drivers
cd ~/Downloads
mkdir -p ixxat_drivers

# Extract the driver package
tar -xzvf ix_usb_can_2.0.520-REL.tgz -C ixxat_drivers
cd ixxat_drivers

# Install build dependencies
sudo apt update
sudo apt install build-essential linux-headers-`uname -r` dkms

# Build the driver
make

# Install the driver
sudo make install

# Load the driver
sudo modprobe ix_usb_can
```

## 3. Configure and Activate the CAN Interface

Set up the CAN interface with 500k baud rate:

```bash
# Set the bitrate to 500k and bring up the interface
sudo ip link set can0 type can bitrate 500000
sudo ip link set up can0
```

Alternate one-line command:

```bash
sudo ip link set can0 type can bitrate 500000 up
```

## 4. Make Sure Driver Loads at Boot

Add the driver to modules list:

```bash
echo "ix_usb_can" | sudo tee -a /etc/modules
```

## 5. Verify the Interface is Working

Check that the interface is up:

```bash
ip -details link show can0
```

You should see "state UP" if everything is working.

## 6. Testing CAN Communication (optional)

Install can-utils for testing:

```bash
sudo apt install can-utils
```

Monitor CAN traffic:

```bash
candump can0
```

Send a test frame:

```bash
cansend can0 123#DEADBEEF
```

In [1]:
!cat serialread.pyread.py

cat: serialread.pyread.py: No such file or directory


In [2]:
!pip list

Package                   Version
------------------------- -----------
anyio                     4.6.2
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     24.3.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
Brotli                    1.0.9
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.3.2
comm                      0.2.1
debugpy                   1.8.11
decorator                 5.1.1
defusedxml                0.7.1
exceptiongroup            1.2.0
executing                 0.8.3
fastjsonschema            2.20.0
h11                       0.14.0
httpcore                  1.0.2
httpx                     0.27.0
idna                      3.7
ipykernel                 6.29.5
ipython                   8.30.0
ipywidgets                8.1.5
jedi                      0.19.2
Jinja2           

In [22]:
#import serial
import can

In [23]:
import struct
def send_one(torque):
    """Sends a single message."""
    #bus = can.Bus(interface='socketcan', channel='can0', bitrate=500000)
    # this uses the default configuration (for example from the config file)
    # see https://python-can.readthedocs.io/en/stable/configuration.html
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        # Using specific buses works similar:
        #use can0 for real, vcan0 for virtual
        # bus = can.Bus(interface='pcan', channel='PCAN_USBBUS1', bitrate=250000)
        # bus = can.Bus(interface='ixxat', channel=0, bitrate=250000)
        # bus = can.Bus(interface='vector', app_name='CANalyzer', channel=0, bitrate=250000)
        # ...
        
        #torque must be smooth function returning value between between 0 and 1
        #torquebyte1, torquebyte2 = struct.pack('<h',torque)
        torque_bytes = struct.pack('<h', torque)
        speed_bytes = struct.pack('<h', 0)
        #speedbyte1 = 0
        #speedbyte2 = 0
        #directioncommand = 1
        #inverterruncommand = 1
        control_byte_4 = 0x00 #forward direction
        control_byte_5 = 0x01 #enable inverter
        #last two bits are torque limits, disregard for us
        torque_limit_bytes = struct.pack('<H', 0) #16-bit, use default eeprom values
        data = torque_bytes + speed_bytes + bytes([control_byte_4, control_byte_5]) + torque_limit_bytes
        command_msg = can.Message(
            arbitration_id=0x0C0, data=data, is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Message sent on {bus.channel_info}")
            print(f"Sent data: {data}")
        except can.CanError:
            print("Message NOT sent")
            
def disableBitch():
    """Sends a single message."""
    #bus = can.Bus(interface='socketcan', channel='can0', bitrate=500000)
    # this uses the default configuration (for example from the config file)
    # see https://python-can.readthedocs.io/en/stable/configuration.html
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        # Using specific buses works similar:
        #use can0 for real, vcan0 for virtual
        # bus = can.Bus(interface='pcan', channel='PCAN_USBBUS1', bitrate=250000)
        # bus = can.Bus(interface='ixxat', channel=0, bitrate=250000)
        # bus = can.Bus(interface='vector', app_name='CANalyzer', channel=0, bitrate=250000)
        # ...
        
        #torque must be smooth function returning value between between 0 and 1
        #torquebyte1, torquebyte2 = struct.pack('<h',torque)
        #speedbyte1 = 0
        #speedbyte2 = 0
        #directioncommand = 1
        #inverterruncommand = 1
        #last two bits are torque limits, disregard for us
        command_msg = can.Message(
            arbitration_id=0x0C0, data=[0, 0, 0, 0, 1, 0, 0, 0], is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Disable Bitch Message sent on {bus.channel_info}")
        except can.CanError:
            print("Message NOT sent")

def enableBitch():
    """Sends a single message."""
    #bus = can.Bus(interface='socketcan', channel='can0', bitrate=500000)
    # this uses the default configuration (for example from the config file)
    # see https://python-can.readthedocs.io/en/stable/configuration.html
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        # Using specific buses works similar:
        #use can0 for real, vcan0 for virtual
        # bus = can.Bus(interface='pcan', channel='PCAN_USBBUS1', bitrate=250000)
        # bus = can.Bus(interface='ixxat', channel=0, bitrate=250000)
        # bus = can.Bus(interface='vector', app_name='CANalyzer', channel=0, bitrate=250000)
        # ...
        
        #torque must be smooth function returning value between between 0 and 1
        #torquebyte1, torquebyte2 = struct.pack('<h',torque)
        #speedbyte1 = 0
        #speedbyte2 = 0
        #directioncommand = 1
        #inverterruncommand = 1
        #last two bits are torque limits, disregard for us
        command_msg = can.Message(
            arbitration_id=0x0C0, data=[0, 0, 0, 0, 1, 1, 0, 0], is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Enable Bitch Message sent on {bus.channel_info}")
        except can.CanError:
            print("Message NOT sent")


In [4]:
import struct
import time
#import numba
x = 300
def torquebytes(value):
    torquebyte1, torquebyte2 = struct.pack('<h',x)
    return torquebyte1, torquebyte2

%timeit torquebytes(301)
#(300).to_bytes(2, byteorder='little')
print(torquebyte1, " and ", torquebyte2)

151 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


NameError: name 'torquebyte1' is not defined

In [6]:
int.from_bytes(b',\x01', byteorder='little')

300

In [11]:
import can

#Bus = can.Bus(channel='can0', interface='socketcan')

can.Bus(channel='can0', interface='socketcan').recv(5)
    

SocketcanBus was not properly shut down


can.Message(timestamp=1748221605.040941, arbitration_id=0xad, is_extended_id=False, channel='can0', dlc=8, data=[0xff, 0xff, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0])

In [ ]:
import time

def time_test():
    timeout = time.time() + 20
    while True:
        print("sending")
        send_one(10)
        if time.time() > timeout:
            print("timed out")
            break

def accel_test():
    for i in range(500):
        print("sending torque value of :", i/10, "Nm")
        send_one(i)
        #time.sleep(0.15)
        print("sent")

In [32]:
disableBitch()

Disable Bitch Message sent on socketcan channel 'can0'


In [34]:
enableBitch()

Enable Bitch Message sent on socketcan channel 'can0'


In [30]:
accel_test()

NameError: name 'accel_test' is not defined

In [11]:
accel_test()

sending torque value of : 0.0 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.1 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.2 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.3 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.4 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.5 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.6 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.7 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.8 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 0.9 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 1.0 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 1.1 Nm
Message sent on socketcan channel 'can0'
sent
sending torque value of : 1.2 Nm
Message sent on soc

In [35]:
#import time
send_one(70)
#time.sleep(2)
#enableBitch()

Message sent on socketcan channel 'can0'
Sent data: b'F\x00\x00\x00\x00\x01\x00\x00'


In [126]:
i = 0
j = 10
while i < 60:
    send_one(i)
    i = i + 1

Message sent on socketcan channel 'can0'
Sent data: b'\x00\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x01\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x02\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x03\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x04\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x05\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x06\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x07\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\x08\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\t\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'can0'
Sent data: b'\n\x00\x00\x00\x01\x01\x00\x00'
Message sent on socketcan channel 'c

In [3]:
import can
import time

# Configure the CAN interface
bus = can.interface.Bus(channel='can0', bustype='socketcan')
 
# Try to send a test message
try:
    message = can.Message(arbitration_id=0x123, data=[0, 1, 2, 3, 4, 5, 6, 7], is_extended_id=False)
    bus.send(message)
    print("Message sent successfully")
except Exception as e:
    print(f"Error sending message: {e}")

# Set up a message listener
print("Listening for messages (press Ctrl+C to stop)...")
try:
    for msg in bus:
        print(f"Received: ID={hex(msg.arbitration_id)}, Data={msg.data}")
        time.sleep(0.1)  # Small delay to prevent flooding the output
except KeyboardInterrupt:
    print("Stopped listening")
finally:
    bus.shutdown()

/tmp/ipykernel_65354/2205870598.py:5: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel='can0', bustype='socketcan')


Message sent successfully
Listening for messages (press Ctrl+C to stop)...
Received: ID=0xad, Data=bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00')
Received: ID=0xac, Data=bytearray(b'\x00\x00\x04\x00\x0f\\\x02\x00')
Received: ID=0xab, Data=bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')
Received: ID=0xaa, Data=bytearray(b'\x04\x00\t\x02\x00\x00\xc0\x00')
Received: ID=0xa8, Data=bytearray(b'"\x00\xedS\x00\x00\xfc\xff')
Received: ID=0xa7, Data=bytearray(b'\xfa\x03\x00\x00\x08\x00\x02\x00')
Received: ID=0xa6, Data=bytearray(b'\x00\x00\xfd\xff\xfe\xff\x00\x00')
Received: ID=0xa5, Data=bytearray(b'\x94\x03\x00\x00\x00\x00n\xfc')
Received: ID=0xa4, Data=bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')
Received: ID=0xa3, Data=bytearray(b'\xe7\x9dw\x1e\xe6\xa5\x97\x1e')
Received: ID=0xad, Data=bytearray(b'\xff\xff\x00\x00\x00\x00\x00\x00')
Received: ID=0xac, Data=bytearray(b'\x00\x00\x04\x00\x13\\\x02\x00')
Received: ID=0xab, Data=bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')
Received: ID=0xaa, D

In [5]:
import can
import struct

def send_one(torque):
    """Sends a single torque command message."""
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        # Convert torque to a 16-bit signed integer value
        # Assuming torque is a float between 0 and 1
        torque_int = int(torque * 32767)  # Scale to max positive 16-bit signed int
        
        # Pack as little-endian 16-bit signed int
        torque_bytes = struct.pack('<h', torque_int)
        
        command_msg = can.Message(
            arbitration_id=0x0C0, 
            data=[torque_bytes[0], torque_bytes[1], 0, 0, 1, 1, 0, 0], 
            is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Torque command sent on {bus.channel_info}")
        except can.CanError:
            print("Message NOT sent")
            
def disableBitch():
    """Disables the inverter run command."""
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        command_msg = can.Message(
            arbitration_id=0x0C0, 
            data=[0, 0, 0, 0, 1, 0, 0, 0], 
            is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Disable message sent on {bus.channel_info}")
        except can.CanError:
            print("Message NOT sent")

def enableBitch():
    """Enables the inverter run command."""
    with can.Bus(interface='socketcan', channel='can0', bitrate=500000) as bus:
        command_msg = can.Message(
            arbitration_id=0x0C0, 
            data=[0, 0, 0, 0, 1, 1, 0, 0], 
            is_extended_id=False
        )

        try:
            bus.send(command_msg)
            print(f"Enable message sent on {bus.channel_info}")
        except can.CanError:
            print("Message NOT sent")

In [10]:
enableBitch()

Enable Bitch Message sent on socketcan channel 'can0'


In [227]:
import can
import struct
import time
from datetime import datetime

def setup_can_bus(channel='can0', bitrate=500000):
    """Setup the CAN bus with the specified channel and bitrate."""
    try:
        bus = can.interface.Bus(channel=channel, bustype='socketcan', bitrate=bitrate)
        print(f"Connected to CAN bus: {channel}")
        return bus
    except Exception as e:
        print(f"Error connecting to CAN bus: {e}")
        return None

def decode_temperature(data):
    """Decode temperature data (signed integer, actual temperature in °C times 10)"""
    return struct.unpack('<h', data)[0] / 10.0  # '<h' for signed 16-bit little-endian

def decode_high_voltage(data):
    """Decode high voltage data (signed integer, actual voltage in Volts times 10)"""
    return struct.unpack('<h', data)[0] / 10.0

def decode_low_voltage(data):
    """Decode low voltage data (signed integer, actual voltage in Volts times 100)"""
    return struct.unpack('<h', data)[0] / 100.0

def decode_current(data):
    """Decode current data (signed integer, actual current in Amps times 10)"""
    return struct.unpack('<h', data)[0] / 10.0

def decode_torque(data):
    """Decode torque data (signed integer, actual torque in N.m times 10)"""
    return struct.unpack('<h', data)[0] / 10.0

def decode_angular_velocity(data):
    """Decode angular velocity (signed integer, actual velocity in RPM)"""
    return struct.unpack('<h', data)[0]

def decode_boolean(data):
    """Decode boolean data (unsigned byte, 1 = true/on, 0 = false/off)"""
    return data[0] > 0

def decode_frequency(data):
    """Decode frequency data (signed integer, actual frequency in Hz times 10)"""
    return struct.unpack('<h', data)[0] / 10.0

def decode_power(data):
    """Decode power data (signed integer, actual power in kW times 10)"""
    return struct.unpack('<h', data)[0] / 10.0

def get_vsm_state_description(state):
    """Get the description for VSM state byte."""
    states = {
        0: "VSM Start State",
        1: "Pre-charge Init State",
        2: "Pre-charge Active State",
        3: "Pre-charge Complete State",
        4: "VSM Wait State",
        5: "VSM Ready State",
        6: "Motor Running State",
        7: "Blink Fault Code State",
        14: "Shutdown in Process",
        15: "Recycle Power State"
    }
    return states.get(state, f"Unknown state ({state})")

def get_inverter_state_description(state):
    """Get the description for Inverter state byte."""
    states = {
        0: "Power on State",
        1: "Stop State",
        2: "Open Loop State",
        3: "Closed Loop State",
        4: "Wait State",
        8: "Idle Run State",
        9: "Idle Stop State"
    }
    return states.get(state, f"Unknown state ({state})")

def decode_fault_codes(data):
    """Decode fault codes from 0x0AB message."""
    post_fault_lo = (data[1] << 8) | data[0]
    post_fault_hi = (data[3] << 8) | data[2]
    run_fault_lo = (data[5] << 8) | data[4]
    run_fault_hi = (data[7] << 8) | data[6]
    
    return {
        "POST Fault Lo": post_fault_lo,
        "POST Fault Hi": post_fault_hi,
        "Run Fault Lo": run_fault_lo,
        "Run Fault Hi": run_fault_hi
    }

def get_post_fault_descriptions(post_fault_lo, post_fault_hi):
    """Get descriptions for POST faults."""
    fault_descriptions = []
    
    # Byte 0 POST faults
    if post_fault_lo & 0x01:
        fault_descriptions.append("Hardware Gate/Desaturation Fault")
    if post_fault_lo & 0x02:
        fault_descriptions.append("HW Over-current Fault")
    if post_fault_lo & 0x04:
        fault_descriptions.append("Accelerator Shorted")
    if post_fault_lo & 0x08:
        fault_descriptions.append("Accelerator Open")
    if post_fault_lo & 0x10:
        fault_descriptions.append("Current Sensor Low")
    if post_fault_lo & 0x20:
        fault_descriptions.append("Current Sensor High")
    if post_fault_lo & 0x40:
        fault_descriptions.append("Module Temperature Low")
    if post_fault_lo & 0x80:
        fault_descriptions.append("Module Temperature High")
        
    # Byte 1 POST faults
    if post_fault_lo & 0x0100:
        fault_descriptions.append("Control PCB Temperature Low")
    if post_fault_lo & 0x0200:
        fault_descriptions.append("Control PCB Temperature High")
    if post_fault_lo & 0x0400:
        fault_descriptions.append("Gate Drive PCB Temperature Low")
    if post_fault_lo & 0x0800:
        fault_descriptions.append("Gate Drive PCB Temperature High")
    if post_fault_lo & 0x1000:
        fault_descriptions.append("5V Sense Voltage Low")
    if post_fault_lo & 0x2000:
        fault_descriptions.append("5V Sense Voltage High")
    if post_fault_lo & 0x4000:
        fault_descriptions.append("12V Sense Voltage Low")
    if post_fault_lo & 0x8000:
        fault_descriptions.append("12V Sense Voltage High")
        
    # Byte 2 POST faults
    if post_fault_hi & 0x01:
        fault_descriptions.append("2.5V Sense Voltage Low")
    if post_fault_hi & 0x02:
        fault_descriptions.append("2.5V Sense Voltage High")
    if post_fault_hi & 0x04:
        fault_descriptions.append("1.5V Sense Voltage Low")
    if post_fault_hi & 0x08:
        fault_descriptions.append("1.5V Sense Voltage High")
    if post_fault_hi & 0x10:
        fault_descriptions.append("DC Bus Voltage High")
    if post_fault_hi & 0x20:
        fault_descriptions.append("DC Bus Voltage Low")
    if post_fault_hi & 0x40:
        fault_descriptions.append("Pre-charge Timeout")
    if post_fault_hi & 0x80:
        fault_descriptions.append("Pre-charge Voltage Failure")
        
    # Byte 3 POST faults
    if post_fault_hi & 0x0100:
        fault_descriptions.append("EEPROM Checksum Invalid")
    if post_fault_hi & 0x0200:
        fault_descriptions.append("EEPROM Data Out of Range")
    if post_fault_hi & 0x0400:
        fault_descriptions.append("EEPROM Update Required")
    if post_fault_hi & 0x0800:
        fault_descriptions.append("Hardware DC Bus Over-Voltage during initialization")
    if post_fault_hi & 0x1000:
        fault_descriptions.append("Gate Driver Initialization")
    if post_fault_hi & 0x2000:
        fault_descriptions.append("Reserved")
    if post_fault_hi & 0x4000:
        fault_descriptions.append("Brake Shorted")
    if post_fault_hi & 0x8000:
        fault_descriptions.append("Brake Open")
    
    return fault_descriptions

def get_run_fault_descriptions(run_fault_lo, run_fault_hi):
    """Get descriptions for RUN faults."""
    fault_descriptions = []
    
    # Byte 4 RUN faults
    if run_fault_lo & 0x01:
        fault_descriptions.append("Motor Over-speed Fault")
    if run_fault_lo & 0x02:
        fault_descriptions.append("Over-current Fault")
    if run_fault_lo & 0x04:
        fault_descriptions.append("Over-voltage Fault")
    if run_fault_lo & 0x08:
        fault_descriptions.append("Inverter Over-temperature Fault")
    if run_fault_lo & 0x10:
        fault_descriptions.append("Accelerator Input Shorted Fault")
    if run_fault_lo & 0x20:
        fault_descriptions.append("Accelerator Input Open Fault")
    if run_fault_lo & 0x40:
        fault_descriptions.append("Direction Command Fault")
    if run_fault_lo & 0x80:
        fault_descriptions.append("Inverter Response Time-out Fault")
        
    # Byte 5 RUN faults
    if run_fault_lo & 0x0100:
        fault_descriptions.append("Hardware Gate/Desaturation Fault")
    if run_fault_lo & 0x0200:
        fault_descriptions.append("Hardware Over-current Fault")
    if run_fault_lo & 0x0400:
        fault_descriptions.append("Under-voltage Fault")
    if run_fault_lo & 0x0800:
        fault_descriptions.append("CAN Command Message Lost Fault")
    if run_fault_lo & 0x1000:
        fault_descriptions.append("Motor Over-temperature Fault")
    if run_fault_lo & 0x2000:
        fault_descriptions.append("Reserved")
    if run_fault_lo & 0x4000:
        fault_descriptions.append("Reserved")
    if run_fault_lo & 0x8000:
        fault_descriptions.append("Reserved")
        
    # Byte 6 RUN faults
    if run_fault_hi & 0x01:
        fault_descriptions.append("Brake Input Shorted Fault")
    if run_fault_hi & 0x02:
        fault_descriptions.append("Brake Input Open Fault")
    if run_fault_hi & 0x04:
        fault_descriptions.append("Module A Over-temperature Fault")
    if run_fault_hi & 0x08:
        fault_descriptions.append("Module B Over-temperature Fault")
    if run_fault_hi & 0x10:
        fault_descriptions.append("Module C Over-temperature Fault")
    if run_fault_hi & 0x20:
        fault_descriptions.append("PCB Over-temperature Fault")
    if run_fault_hi & 0x40:
        fault_descriptions.append("Gate Drive Board 1 Over-temperature Fault")
    if run_fault_hi & 0x80:
        fault_descriptions.append("Gate Drive Board 2 Over-temperature Fault")
        
    # Byte 7 RUN faults
    if run_fault_hi & 0x0100:
        fault_descriptions.append("Gate Drive Board 3 Over-temperature Fault")
    if run_fault_hi & 0x0200:
        fault_descriptions.append("Current Sensor Fault")
    if run_fault_hi & 0x0400:
        fault_descriptions.append("Gate Driver Over-Voltage")
    if run_fault_hi & 0x0800:
        fault_descriptions.append("Reserved (or Hardware DC Bus Over-Voltage for Gen 3)")
    if run_fault_hi & 0x1000:
        fault_descriptions.append("Hardware DC Bus Over-voltage Fault")
    if run_fault_hi & 0x2000:
        fault_descriptions.append("Reserved")
    if run_fault_hi & 0x4000:
        fault_descriptions.append("Resolver Not Connected")
    if run_fault_hi & 0x8000:
        fault_descriptions.append("Reserved")
    
    return fault_descriptions


def parse_cascadia_can_message(msg):
    """Parse CAN messages from Cascadia Motion motor controller."""
    
    # Adjust base_id based on your CAN ID offset (default 0x0A0)
    base_id = 0x0A0
    
    # Filter messages based on ID
    if msg.arbitration_id >= base_id and msg.arbitration_id <= base_id + 0x2F:
        # Get the message type by its offset from base ID
        msg_type = msg.arbitration_id - base_id
        
        # Map message types to human-readable formats
        if msg_type == 0x00:  # 0x0A0 - Temperatures #1
            module_a_temp = decode_temperature(msg.data[0:2])
            module_b_temp = decode_temperature(msg.data[2:4])
            module_c_temp = decode_temperature(msg.data[4:6])
            gate_driver_temp = decode_temperature(msg.data[6:8])
            return {
                "message": "Temperatures #1",
                "Module A Temperature": f"{module_a_temp:.1f} °C",
                "Module B Temperature": f"{module_b_temp:.1f} °C",
                "Module C Temperature": f"{module_c_temp:.1f} °C",
                "Gate Driver Board Temperature": f"{gate_driver_temp:.1f} °C"
            }
            
        elif msg_type == 0x07:  # 0x0A7 - Voltage Information
            dc_bus_voltage = decode_high_voltage(msg.data[0:2])
            output_voltage = decode_high_voltage(msg.data[2:4])
            vab_vd_voltage = decode_high_voltage(msg.data[4:6])
            vbc_vq_voltage = decode_high_voltage(msg.data[6:8])
            return {
                "message": "Voltage Information",
                "DC Bus Voltage": f"{dc_bus_voltage:.1f} V",
                "Output Voltage": f"{output_voltage:.1f} V",
                "VAB/Vd Voltage": f"{vab_vd_voltage:.1f} V",
                "VBC/Vq Voltage": f"{vbc_vq_voltage:.1f} V"
            }
            
        elif msg_type == 0x06:  # 0x0A6 - Current Information
            phase_a_current = decode_current(msg.data[0:2])
            phase_b_current = decode_current(msg.data[2:4])
            phase_c_current = decode_current(msg.data[4:6])
            dc_bus_current = decode_current(msg.data[6:8])
            return {
                "message": "Current Information",
                "Phase A Current": f"{phase_a_current:.1f} A",
                "Phase B Current": f"{phase_b_current:.1f} A",
                "Phase C Current": f"{phase_c_current:.1f} A",
                "DC Bus Current": f"{dc_bus_current:.1f} A"
            }
            
        elif msg_type == 0x05:  # 0x0A5 - Motor Position Information
            motor_angle = decode_temperature(msg.data[0:2])  # Angle uses same format as temperature
            motor_speed = decode_angular_velocity(msg.data[2:4])
            electrical_output_freq = decode_frequency(msg.data[4:6])
            delta_resolver_filtered = decode_temperature(msg.data[6:8])
            return {
                "message": "Motor Position Information",
                "Motor Angle": f"{motor_angle:.1f}°",
                "Motor Speed": f"{motor_speed} RPM",
                "Electrical Output Frequency": f"{electrical_output_freq:.1f} Hz",
                "Delta Resolver Filtered": f"{delta_resolver_filtered:.1f}°"
            }
            
        elif msg_type == 0x0C:  # 0x0AC - Torque & Timer Information
            commanded_torque = decode_torque(msg.data[0:2])
            torque_feedback = decode_torque(msg.data[2:4])
            power_on_timer = struct.unpack('<I', msg.data[4:8])[0] * 0.003
            return {
                "message": "Torque & Timer Information",
                "Commanded Torque": f"{commanded_torque:.1f} Nm",
                "Torque Feedback": f"{torque_feedback:.1f} Nm",
                "Power On Timer": f"{power_on_timer:.1f} sec"
            }
        
        elif msg_type == 0x0A:  # 0x0AA - Internal States
            vsm_state = msg.data[0]
            pwm_frequency = msg.data[1]
            inverter_state = msg.data[2]
            relay_state = msg.data[3]
            
            # Byte 4 contains several bit flags
            inverter_run_mode = (msg.data[4] & 0x01) > 0  # Bit 0
            self_sensing_assist = (msg.data[4] & 0x02) > 0  # Bit 1
            
            # Bits 5-7 of byte 4 are Inverter Active Discharge State
            discharge_state = (msg.data[4] >> 5) & 0x07
            discharge_states = {
                0: "Discharge Disabled",
                1: "Discharge Enabled, waiting",
                2: "Performing Speed Check",
                3: "Discharge Actively occurring",
                4: "Discharge Completed"
            }
            discharge_description = discharge_states.get(discharge_state, f"Unknown ({discharge_state})")
            
            # Byte 5 contains more flags
            inverter_command_mode = (msg.data[5] & 0x01) > 0  # Bit 0
            rolling_counter = (msg.data[5] >> 4) & 0x0F  # Bits 4-7
            
            # Byte 6 contains more flags
            inverter_enable_state = (msg.data[6] & 0x01) > 0  # Bit 0
            burst_model_mode = (msg.data[6] & 0x02) > 0  # Bit 1
            start_mode_active = (msg.data[6] & 0x40) > 0  # Bit 6
            inverter_enable_lockout = (msg.data[6] & 0x80) > 0  # Bit 7
            
            # Byte 7 contains more flags
            direction_command = (msg.data[7] & 0x01) > 0  # Bit 0
            bms_active = (msg.data[7] & 0x02) > 0  # Bit 1
            bms_limiting_torque = (msg.data[7] & 0x04) > 0  # Bit 2
            limit_max_speed = (msg.data[7] & 0x08) > 0  # Bit 3
            limit_hot_spot = (msg.data[7] & 0x10) > 0  # Bit 4
            low_speed_limiting = (msg.data[7] & 0x20) > 0  # Bit 5
            coolant_temp_limiting = (msg.data[7] & 0x40) > 0  # Bit 6
            limit_stall_burst_model = (msg.data[7] & 0x80) > 0  # Bit 7
            
            return {
                "message": "Internal States",
                "VSM State": get_vsm_state_description(vsm_state),
                "PWM Frequency": f"{pwm_frequency} kHz",
                "Inverter State": get_inverter_state_description(inverter_state),
                "Relay State": f"0x{relay_state:02X}",
                "Inverter Run Mode": "Speed Mode" if inverter_run_mode else "Torque Mode",
                "Self-Sensing Assist": "Enabled" if self_sensing_assist else "Disabled",
                "Inverter Active Discharge State": discharge_description,
                "Inverter Command Mode": "VSM Mode" if inverter_command_mode else "CAN Mode",
                "Rolling Counter Value": rolling_counter,
                "Inverter Enable State": "Enabled" if inverter_enable_state else "Disabled",
                "Burst Model Mode": "High Speed" if burst_model_mode else "Stall",
                "Start Mode Active": "Active" if start_mode_active else "Inactive",
                "Inverter Enable Lockout": "Locked" if inverter_enable_lockout else "Unlocked",
                "Direction Command": "Forward" if direction_command else "Reverse",
                "BMS Active": "Active" if bms_active else "Inactive",
                "BMS Limiting Torque": "Yes" if bms_limiting_torque else "No",
                "Limit Max Speed": "Active" if limit_max_speed else "Inactive",
                "Limit Hot Spot": "Active" if limit_hot_spot else "Inactive",
                "Low Speed Limiting": "Active" if low_speed_limiting else "Inactive",
                "Coolant Temp Limiting": "Active" if coolant_temp_limiting else "Inactive",
                "Limit Stall Burst Model": "Active" if limit_stall_burst_model else "Inactive"
            }
        
        """Parse fault code messages (0x0AB)."""
        elif msg_type == 0x0B:
            fault_codes = decode_fault_codes(msg.data)
            post_fault_lo = fault_codes["POST Fault Lo"]
            post_fault_hi = fault_codes["POST Fault Hi"]
            run_fault_lo = fault_codes["Run Fault Lo"]
            run_fault_hi = fault_codes["Run Fault Hi"]
            
            post_faults = get_post_fault_descriptions(post_fault_lo, post_fault_hi)
            run_faults = get_run_fault_descriptions(run_fault_lo, run_fault_hi)
            
            result = {
                "message": "Fault Codes",
                "POST Fault Lo": f"0x{post_fault_lo:04X}",
                "POST Fault Hi": f"0x{post_fault_hi:04X}",
                "Run Fault Lo": f"0x{run_fault_lo:04X}",
                "Run Fault Hi": f"0x{run_fault_hi:04X}"
            }
            
            if post_faults:
                result["POST Faults Present"] = ", ".join(post_faults)
            else:
                result["POST Faults Present"] = "None"
                
            if run_faults:
                result["Run Faults Present"] = ", ".join(run_faults)
            else:
                result["Run Faults Present"] = "None"
            
            return result
            # Add more message type handlers based on your needs
        
    return None

def main():
    # Setup the CAN bus (adjust channel and bitrate as needed)
    bus = setup_can_bus(channel='can0', bitrate=500000)
    if not bus:
        return
    
    print("Listening for CAN messages. Press Ctrl+C to exit.")
    
    try:
        while True:
            msg = bus.recv(1.0)  # Timeout in seconds
            if msg:
                result = parse_cascadia_can_message(msg)
                if result:
                    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
                    print(f"\n[{timestamp}] ID: 0x{msg.arbitration_id:X} - {result['message']}")
                    for key, value in result.items():
                        if key != "message":
                            print(f"  {key}: {value}")
    
    except KeyboardInterrupt:
        print("\nExiting...")
    finally:
        if bus:
            bus.shutdown()

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (1001609400.py, line 394)